# Differential Evolution

In [909]:
import numpy as np
from numpy.random import choice, uniform

In [910]:
def init_population(pop_size, var_count, min_bound, max_bound):
    return uniform(low=min_bound, high=max_bound, size=(pop_size, var_count))

In [911]:
population = init_population(10, 2, -30, 30)
population

array([[  8.36722255,  -6.2380485 ],
       [ -0.34300279,  27.61530321],
       [ 21.89709055,  25.09431824],
       [-26.70828167,  14.30519681],
       [ -0.97720696, -21.71791873],
       [  5.74359326,  24.01643621],
       [  3.71848055, -23.25409724],
       [ -4.30035326,  27.21243208],
       [-26.09337699,  29.60765792],
       [  7.94309051,  -5.17800214]])

In [912]:
def calc_fitness_scores(population):
    scores = []

    for vector in population:
        # Rosenbrock's Function
        score = 100 * (vector[1] - vector[0]**2)**2 + (vector[0] - 1)**2
        scores.append(score)

    return scores

In [913]:
scores = calc_fitness_scores(population)
scores

[581437.0674469668,
 75613.89184751226,
 20647305.43599856,
 48864658.18699578,
 51409.731891048745,
 8072.946904738247,
 137508.8914368109,
 7630.8768186944535,
 42414258.41288832,
 466136.90433785855]

In [914]:
def differential_mutation(idx, population, min_bound, max_bound, scaling_factor = 0.5):
    used_idx = [idx]
    used_idx_no = 3

    while len(used_idx) <= used_idx_no:
        idx_selected = False
        while idx_selected == False:
            vector_idx = choice(np.array(range(len(population))), 1, replace=False)[0]
            if vector_idx not in used_idx:
                idx_selected = True
                used_idx.append(vector_idx)

    target_vector = used_idx[1]
    ran_select_1 = used_idx[2]
    ran_select_2 = used_idx[3]

    trial_vector = []
    trial_vec_loc_1 = population[target_vector][0] + scaling_factor * (population[ran_select_1][0] - population[ran_select_2][0])
    if trial_vec_loc_1 < min_bound:
        trial_vector.append(min_bound)
    elif trial_vec_loc_1 > max_bound:
        trial_vector.append(max_bound)
    else:
        trial_vector.append(trial_vec_loc_1)
    trial_vec_loc_2 = population[target_vector][1] + scaling_factor * (population[ran_select_1][1] - population[ran_select_2][1])
    if trial_vec_loc_2 < min_bound:
        trial_vector.append(min_bound)
    elif trial_vec_loc_2 > max_bound:
        trial_vector.append(max_bound)
    else:
        trial_vector.append(trial_vec_loc_2)

    return trial_vector

In [915]:
trial_vector = differential_mutation(1, population, -30, 30)
trial_vector

[-27.80803029173588, 28.90241598413484]

In [916]:
def crossover(idx, population, trial_vector, crossover_prob = 0.7):
    parent_vector = population[idx]
    offspring_vector = []
    var_len = len(parent_vector)
    trial_vec_select_idx = list(choice(np.array(range(var_len)), 1, replace=False))
    for i in range(var_len):
        if uniform(0, 1) < crossover_prob and i not in trial_vec_select_idx:
            trial_vec_select_idx.append(i)
    for j in range(var_len):
        if j in trial_vec_select_idx:
            offspring_vector.append(trial_vector[j])
        else:
            offspring_vector.append(parent_vector[j])

    return offspring_vector

In [917]:
offspring_vector = crossover(1, population, trial_vector, crossover_prob = 0.7)
offspring_vector

[-27.80803029173588, 28.90241598413484]

In [918]:
def selection(idx, offspring_vector, population, scores):
    sub_pop = [population[idx], offspring_vector]
    sub_scores = calc_fitness_scores(sub_pop)
    if sub_scores[1] < sub_scores[0]:
        population[idx] = offspring_vector
        scores[idx] = sub_scores[1]

    return

In [919]:
selection(1, offspring_vector, population, scores)
population

array([[  8.36722255,  -6.2380485 ],
       [ -0.34300279,  27.61530321],
       [ 21.89709055,  25.09431824],
       [-26.70828167,  14.30519681],
       [ -0.97720696, -21.71791873],
       [  5.74359326,  24.01643621],
       [  3.71848055, -23.25409724],
       [ -4.30035326,  27.21243208],
       [-26.09337699,  29.60765792],
       [  7.94309051,  -5.17800214]])

In [920]:
scores

[581437.0674469668,
 75613.89184751226,
 20647305.43599856,
 48864658.18699578,
 51409.731891048745,
 8072.946904738247,
 137508.8914368109,
 7630.8768186944535,
 42414258.41288832,
 466136.90433785855]

In [921]:
def differential_evolution(var_count, min_bound, max_bound, pop_size, generations, scaling_factor = 0.5, crossover_prob = 0.7):
    population = init_population(pop_size, var_count, min_bound, max_bound)
    scores = calc_fitness_scores(population)
    for i in range(generations):
        print(f"******************************* Generation {i + 1} *******************************")
        print("Population-------------------------------------------------------Scores")
        for idx, vector in enumerate(population):
            trial_vector = differential_mutation(idx, population, min_bound, max_bound)
            offspring_vector = crossover(idx, population, trial_vector, 0.7)
            selection(idx, offspring_vector, population, scores)
            print(str(population[idx])+"                              "+str(scores[idx]))
        best_idx = scores.index(min(scores))
        print("===============================================================================")
        print(f"Best Vector: {population[best_idx]}----------------Score: {scores[best_idx]}")
        print("===============================================================================")

In [922]:
differential_evolution(2, -30, 30, 20, 100)

******************************* Generation 1 *******************************
Population-------------------------------------------------------Scores
[15.3090901   7.71236267]                              5137493.403851249
[  7.64548175 -19.58919578]                              609108.7016364188
[22.73560965 22.03431859]                              24490463.16736796
[-7.69567549 30.        ]                              85476.44946494894
[ -1.78798951 -11.16168714]                              20624.69391896844
[27.190737   22.92407369]                              51325316.39572815
[  5.07740019 -30.        ]                              311157.3835855086
[ 5.24878536 24.60339314]                              886.1527006745838
[ 10.49028313 -29.87906869]                              1957993.6751537488
[-2.363033   28.86730691]                              54222.89747179775
[-3.95439474 27.02418736]                              12990.808066983996
[15.80092448 -0.03205276]             